<a href="https://colab.research.google.com/github/yk-Jeong/Kaggle-Studies/blob/main/7th_cassava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(1/14~1/20) 스터디 7주차: https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-tensorflow-training/notebook


In [1]:
!pip install --queit efficientnet


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --queit


In [2]:
!pip install -q kaggle

In [3]:
!pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-rkvb2s69
  Running command git clone -q https://github.com/qubvel/segmentation_models /tmp/pip-req-build-rkvb2s69
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 2.6 MB/s 
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-py3-none-any.whl size=33810 sha256=757a281e0bd63420e302196107995f276a4c477758d2c96301eef3de5d51e7fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-y9o0mspt/wheels/02/cd/18/61c0bbb8766acfec68f9d20618886b7b38dfeeb95865b6ba00
Successfully built segmentation-models


In [4]:
!mkdir -p ~/.kaggle !cp kaggle.json ~/.kaggle/ 
!cmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘kaggle.json’: File exists
/bin/bash: cmod: command not found


In [5]:
!kaggle datasets download -c cassava-leaf-disease-classification

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


1. lib load and memory setting 

In [6]:
import math, os, re, warnings, random

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
#from kaggle_datasets import KaggleDatasets
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

from tensorflow.keras import optimizers, applications, Sequential, losses, metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

import efficientnet.tfkeras as efn

In [7]:
def seed_everything(seed = 0):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')

In [8]:
#TPU or GPU detection

try: 
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print(f'Running on TPU {tpu.master()}')
except ValueError:
  tpu = None

if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

REPLICAS: 1


In [9]:
#params

BATCH_SIZE = 16 * REPLICAS
LEARNING_RATE = 3e-5 * REPLICAS
EPOCHS = 30
HEIGHT = 512
WIDTH = 512
CHANNELS = 3
ES_PATIENCE = 10
N_FOLDS = 5

2. load data

In [11]:
def count_data_items(filenames):
  n = [int(re.compile(r'-([0-9]*)\.').search(filename).group(1)) for filename in filenames]
  return np.sum(n)

database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'
train = pd.read_csv(f'{database_base_path}train.csv')
print(f'Train samples: {len(train)}')

GCS_PATH = KaggleDatasets().get_gcs_path(f'cassava-leaf-disease-tfrecords-center-{HEIGHT}x{WIDTH}') 
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/*.tfrec')
NUM_TRAINING_IMAGES= count_data_items(TRAINING_FILENAMES)

print(f'GCS: train images: {NUM_TRAINING_IMAGES}')
display(train.head())

FileNotFoundError: ignored

In [12]:
CLASSES = ['Cassava Bacterial Blight', 
           'Cassava Brown Streak Disease', 
           'Cassava Green Mottle', 
           'Cassava Mosaic Disease', 
           'Healthy']

Augmentation 

In [13]:
#학습률 향상을 위해 이미지를 리터칭한다 

def data_augment(image, label):
  p_rotation = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_shear = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_crop = tf.random.uniform([], 0, 1.0, dtype = tf.float32)

#shear

  if p_shear > .2:
    if p_shear > .6:
      image = transform_shear(image, HEIGHT, shear = 20.)
    else:
      image - transform_shear(image, HEIGHT, shear = -20.)
  
#rotation
  if p_rotation > .2:
    if p_rotation > .6:
      image = transform_rotation(iamge, HEIGHT, rotation = 45.)
    else:
      image = transform_rotation(iamge, HEIGHT, rotation = -45.)

#flips
image = tf.